# Data Pre-Processing

Import essential packages

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Read in `Results.csv`

In [2]:
df = pd.read_csv("./data/Results.csv", sep=";", decimal=",") # Separator is semi-colon and decimal point is comma

Convert the `time` field to `Pandas DateTime`

In [3]:
df.time      = pd.to_datetime(df.time)   # Convert datefield to datetime
df           = df.sort_values(by='time') # Sort values on the time column
df           = df.dropna()               # Remove NaNs (there's only one)

Separate the dataframe into device-unique instances (i.e. on sensor)

In [4]:
devices      = sorted(df.tag.unique())           # Create a list of sorted device tags, there are 4 sensors
device_names = [name[15:22] for name in devices] # Make a list of device unique names, i.e. 0001_PI, 0002_PI, ...
grouped      = df.groupby(df.tag)                # Create a groupby generator on the .tag column (sensor names)
dfs          = {}                                # Empty dictionary to hold device-unique dataframes

# For every unique device
for i, device in enumerate(devices):   
    # Add device unique group to dictionary. The key is device unique, i.e. 0001_PI, 0002_PI, ...
    dfs["{}".format(device_names[i])] = grouped.get_group(device).copy()

This should be fine, but I'm not all too fond of this stacked format we have where the `average`, `minimum` and `maximum` values are in seperate rows.<br> 
I'll now *unstack* the dataframe so that these appear in different columns, by using `.pivot()`.

In [5]:
for key, dataframe in dfs.items():
    dfs[key] = dataframe.pivot(index="time", columns="mode", values=['value','quality']).copy()

The multi-index column names are a bit awkward, e.g. `maximum:PT10M` etc.<br>
I'll create a cleaner version and apply to all of them

In [8]:
# Create a list of easily interpretable tuples
new_columns = [('value','avg'),
               ('value','max'),
               ('value','min'),
               ('quality','q_avg'),
               ('quality','q_max'),
               ('quality','q_min')]

# And generate a multi-index out of it
column_idx  = pd.MultiIndex.from_tuples(new_columns)

# Apply it to all the device-unique dataframes in the dictionary
for key, dataframe in dfs.items():
    dataframe.columns = column_idx
    dataframe.columns = dataframe.columns.droplevel()

Now, I want to check if all the `quality` values of the frames are the same, if so, then some columns are redundant

In [10]:
dfs['0001_PI'].q_min.equals(dfs['0001_PI'].q_avg)

False

Hmm, unfortunately it seems **not**. <br>
Let's check where this mismatch is happening.

In [37]:
# Find the index numbers where the quality of the average value is not equal to the quality of the minimum one
mismatches = np.where(dfs['0001_PI'].q_min != dfs['0001_PI'].q_avg)[0]

# Display the row values where this is happening
dfs['0001_PI'].iloc[mismatches[0]]

avg         NaN
max        5.68
min        5.32
q_avg       NaN
q_max    100.00
q_min    100.00
Name: 2020-03-21 17:01:40, dtype: float64

Okay, strangely enough, there seem to be `NaNs` present in the average columns which are not reflected in the `max` and `min` ones.<br>
Maybe we need a clarification if there is some minimum number of measurements required to generate an average measurement.